# Life-Cycle Management of Service by the registry
### Three lifecycle options
- Transient: A new instance is created every time the service is resolved. This is useful for stateless services or when you always want a fresh instance.
- Singleton: Only one instance is created and reused for the entire lifetime of the container. This is useful for services that are expensive to create or need to maintain state across the entire application.
- Scoped: A new instance is created for each scope, and that instance is reused within that scope. This is useful for services that should have a lifetime tied to a specific context (e.g., a web request).
### Lifecycle behavior
- Transient services always create new instances.
- Singleton services reuse the same instance.
- Scoped services create new instances for each scope but reuse them within the scope.

In [1]:
# Adding service life-cycle management

from typing import Callable, Any, Dict
from abc import ABC, abstractmethod
from enum import Enum, auto

class Lifecycle(Enum):
    TRANSIENT = auto()
    SINGLETON = auto()
    SCOPED = auto()

class DIContainer:
    def __init__(self):
        self._factories: Dict[str, Callable[[], Any]] = {}
        self._lifecycles: Dict[str, Lifecycle] = {}
        self._instances: Dict[str, Any] = {}
        self._scoped_instances: Dict[str, Dict[int, Any]] = {}
        self._current_scope: int = 0

    def register(self, name: str, factory: Callable[[], Any], lifecycle: Lifecycle = Lifecycle.TRANSIENT):
        self._factories[name] = factory
        self._lifecycles[name] = lifecycle

    def resolve(self, name: str) -> Any:
        if name not in self._factories:
            raise KeyError(f"Service '{name}' not registered.")

        lifecycle = self._lifecycles[name]

        if lifecycle == Lifecycle.SINGLETON:
            if name not in self._instances:
                self._instances[name] = self._factories[name]()
            return self._instances[name]
        elif lifecycle == Lifecycle.SCOPED:
            if self._current_scope == 0:
                raise RuntimeError("No active scope")
            if name not in self._scoped_instances:
                self._scoped_instances[name] = {}
            if self._current_scope not in self._scoped_instances[name]:
                self._scoped_instances[name][self._current_scope] = self._factories[name]()
            return self._scoped_instances[name][self._current_scope]
        else:  # TRANSIENT
            return self._factories[name]()

    def begin_scope(self):
        self._current_scope += 1
        return self._current_scope

    def end_scope(self, scope_id: int):
        if scope_id != self._current_scope:
            raise ValueError("Cannot end a scope that is not current")
        for scoped_instances in self._scoped_instances.values():
            scoped_instances.pop(scope_id, None)
        self._current_scope -= 1

class ILogger(ABC):
    @abstractmethod
    def log(self, message: str):
        pass

class ConsoleLogger(ILogger):
    def __init__(self):
        self.log_count = 0

    def log(self, message: str):
        self.log_count += 1
        print(f"[LOG {self.log_count}] {message}")

class DataService:
    def __init__(self, logger: ILogger):
        self.logger = logger

    def process_data(self):
        self.logger.log("Processing data...")

# Usage example
container = DIContainer()

# Register services with different lifecycles
container.register("transient_logger", lambda: ConsoleLogger(), Lifecycle.TRANSIENT)
container.register("singleton_logger", lambda: ConsoleLogger(), Lifecycle.SINGLETON)
container.register("scoped_logger", lambda: ConsoleLogger(), Lifecycle.SCOPED)

container.register("transient_service", lambda: DataService(container.resolve("transient_logger")), Lifecycle.TRANSIENT)
container.register("singleton_service", lambda: DataService(container.resolve("singleton_logger")), Lifecycle.SINGLETON)
container.register("scoped_service", lambda: DataService(container.resolve("scoped_logger")), Lifecycle.SCOPED)

# Demonstrate transient lifecycle
print("Transient Lifecycle:")
container.resolve("transient_service").process_data()
container.resolve("transient_service").process_data()

# Demonstrate singleton lifecycle
print("\nSingleton Lifecycle:")
container.resolve("singleton_service").process_data()
container.resolve("singleton_service").process_data()

# Demonstrate scoped lifecycle
print("\nScoped Lifecycle:")
scope1 = container.begin_scope()
container.resolve("scoped_service").process_data()
container.resolve("scoped_service").process_data()
container.end_scope(scope1)

print("\nNew Scope:")
scope2 = container.begin_scope()
container.resolve("scoped_service").process_data()
container.resolve("scoped_service").process_data()
container.end_scope(scope2)

Transient Lifecycle:
[LOG 1] Processing data...
[LOG 1] Processing data...

Singleton Lifecycle:
[LOG 1] Processing data...
[LOG 2] Processing data...

Scoped Lifecycle:
[LOG 1] Processing data...
[LOG 2] Processing data...

New Scope:
[LOG 1] Processing data...
[LOG 2] Processing data...
